### Import all required libraries

In [5]:
from music21 import *
import itertools
from itertools import tee
from collections import OrderedDict
import pandas as pd
import os
import time
import numpy as np
import matplotlib.pyplot as plt
import math
from numpy.random import randn
from matplotlib import cm
import matplotlib.pyplot as plt
import seaborn as sns
from collections import Counter
from scipy.ndimage import gaussian_filter
from tqdm import tqdm

### Transform folder of note csvs to one matrix.

In [8]:
x = 0
a = ['C', 'C#', 'D', 'D#', 'E', 'F', 'F#', 'G', 'G#', 'A', 'A#', 'B']

c = list(dict.fromkeys(list(map(' '.join, itertools.chain(
    itertools.product(a, a), itertools.product(a, a))))))


df2 = pd.DataFrame(columns=c, index=c)


for col in df2.columns:
    df2[col].values[:] = 0

totals = []

midi = pd.read_csv('data/midi.csv')
for index, row in midi.iterrows():
    try:
        filepath = 'data/pieces/' + row['File Path'] + '.csv'
        df1 = pd.read_csv(filepath, index_col=0)
        totals.append(df1.to_numpy().sum())
        df2 = pd.concat([df1, df2]).groupby(level=0).sum()
    except:
        print(f'Failed with {filepath}')
df2 = df2.reindex(a)

df4 = df2

for note in a:
    df4.loc[note] = df2.loc[note] / df2.sum(axis=1)[note]
df4

,C,C#,D,D#,E,F,F#,G,G#,A,A#,B
C,0.230392,0.045169,0.074459,0.103465,0.082472,0.093224,0.026560,0.076252,0.058811,0.071034,0.078986,0.059176
C#,0.063325,0.198458,0.076782,0.067626,0.109404,0.080503,0.075915,0.035570,0.077349,0.063045,0.079100,0.072924
D,0.078874,0.051870,0.234962,0.053102,0.076919,0.105704,0.067918,0.091571,0.028713,0.072146,0.071305,0.066918
D#,0.092664,0.069238,0.066104,0.212696,0.052214,0.075579,0.082705,0.088110,0.092892,0.039098,0.083735,0.044965
E,0.065798,0.070457,0.094207,0.043543,0.201287,0.067703,0.063672,0.111162,0.070080,0.107284,0.032067,0.072740
F,0.074755,0.053152,0.085016,0.074755,0.070371,0.229132,0.040593,0.077890,0.085306,0.081955,0.096462,0.030614
F#,0.037358,0.071034,0.068259,0.079686,0.087883,0.052551,0.196112,0.069888,0.066129,0.120243,0.063907,0.086951
G,0.100682,0.030608,0.072721,0.062858,0.075099,0.084511,0.051208,0.234339,0.047855,0.074292,0.095200,0.070627
G#,0.099421,0.084052,0.040249,0.085689,0.062744,0.084161,0.067910,0.061332,0.195484,0.067240,0.064119,0.087599
A,0.097382,0.065282,0.102427,0.034302,0.075262,0.060428,0.068738,0.077996,0.051420,0.238601,0.060352,0.067809


### correctNote()
Function to correct sharps and flats to the same note

In [9]:
def correctNote(note):
    note_letter = note[:1]

    note_step = list(map(lambda i:i, note[1:]))
    
    note_position = a.index(note_letter)
    
    for x in note_step:
        if x == '-':
            if note_position == 0:
                note_position = 11
            else:
                note_position = note_position - 1
        if x == '#':
            if note_position == 11:
                note_position = 0
            else:
                note_position = note_position + 1
    return a[note_position]

In [10]:
def pairwise(iterable):
    a, b = tee(iterable)
    next(b, None)
    return zip(a, b)

In [14]:
def getSong(filepath):
    
    global c_dt
    
    song_dict = { i : 0 for i in c }
    filepath_extended = 'data/midi/' + filepath + '.mid'
    s = converter.parse(filepath_extended)
    
    pair_count = 0


    recurseIter = s.recurse()

    chopin_notes = []
    x = 0
    for i in recurseIter.notes:
        
        if 'Chord' in str(i):
            chord_pitches = i.pitches
            for i in chord_pitches:
                chopin_notes.append(correctNote(i.name))
        else:
            chopin_notes.append(correctNote(i.name))
            

    bigram_list_prob = []
    for v, w in pairwise(chopin_notes):
        bigram_prob = big_data[w][v]
        bigram_list_prob.append(bigram_prob)
        
        
    pair_count = len(bigram_list_prob)
    
    melodic_originality = 1-sum(bigram_list_prob)/pair_count
    

    return song_dict, pair_count, bigram_list_prob, melodic_originality

### analyzePiece()
Calculate the melodic originality for a given piece.

In [15]:
big_data = df4
def analyzePiece(song):
    song_dict, pair_count, bigram_list_prob, melodic_originality = getSong(song)
    df2 = pd.DataFrame(columns=a, index=a)

    x = 0
    for key,value in song_dict.items():
        if value > 0:
            frequency = value/pair_count
            df.loc[x] = [key] + [frequency, value]
            x = x + 1
            notes = key.split(' ')

            note_1 = notes[0]
            note_2 = notes[1]
            df2.loc[note_1,note_2] = value
            print(f'Note: {key}, Frequency:   {frequency}, Value: {value}')
    dataframe_1 = df2.fillna(0)

    dataframe_2 = dataframe_1.div(pair_count)
    
    
    total_prob = sum(bigram_list_prob)

    test = pair_count

    melodic_originality = 1-(total_prob/test)
    return melodic_originality

Iterate through the piece dataframe to calculate the melodic originality for each peice.

In [24]:
midi["Melodic Originality"] = ""
for index, row in midi.iterrows():
        file_path = row['File Path']
        melodic_originality = analyzePiece(file_path)
        print(f'[{index+1}] Piece: {file_path}, Originality: {melodic_originality}')
        midi.iat[index, 3] = melodic_originality
        midi.to_csv('data/result.csv', index=False)
midi

[0] Piece: consolation_6_(c)halstead, Originality: 0.893871001657552
[1] Piece: valse_18_(c)dery, Originality: 0.8997888569542055
[2] Piece: mazurka_6_2_(c)yogore, Originality: 0.9072540968726254
[3] Piece: sonata_D958_4_(c)bucknall, Originality: 0.8995212867550562
[4] Piece: strings_12-1_(c)onclassical, Originality: 0.8837578677066256
[5] Piece: nocturne_62_2_(c)unknown, Originality: 0.9100904858468507
[6] Piece: winterreise_10_(c)yogore, Originality: 0.9017066981978862
[7] Piece: klavierstucke_119_3_(c)lefeldt, Originality: 0.8966998101883872
[8] Piece: sonata_58_4_(c)finley, Originality: 0.9042989069736012


KeyboardInterrupt: 

In [25]:
repetoire_df = pd.read_csv('data/result.csv')
pd.set_option('display.max_rows', 50)

df2 = pd.DataFrame(columns=a, index=a)

repetoire_df

,File Path,Piece Tital,Composer,Melodic Originality
0,consolation_6_(c)halstead,Consolation No.6 in E Major,Liszt,0.893871
1,valse_18_(c)dery,Grande Valse Brillante in E-flat major,Chopin,0.899789
2,mazurka_6_2_(c)yogore,Mazurka Op.6 No.2,Chopin,0.907254
3,sonata_D958_4_(c)bucknall,"Piano Sonata No. 19 in C Minor, D. 958 - 4, Al...",Schubert,0.899521
4,strings_12-1_(c)onclassical,"String Quartet No.12, D 703",Schubert,0.883758
...,...,...,...,...
426,strings_11-1_(c)onclassical,"Johannes Brahms - Serenade No.1 in D-major, Op...",Brahms,NaN
427,valse_70_2_(c)dery,"Chopin Valse Op.70 No.2 Paul Barton, FEURICH p...",Chopin,NaN
428,hungarian_dance_14_(c)yogore,Brahms: Hungarian Dance No.14 In D Minor,Brahms,NaN
429,symphony_5_1_(c)lucarelli,"Beethoven -5th Symphony, 1st movement: Allegro...",Beethoven,NaN
